# *Covid19* - Variation in CO2 Emission from Europe’s electricity system

In [ ]:
import sys
import os
from qgis.core import *
from qgis.PyQt.QtGui import *
from qgis.PyQt.QtCore import *

## QGIS inicialization

In [ ]:
QgsApplication.setPrefixPath(r'C:\Program Files\QGIS 3.12\apps\qgis', True)
qgs = QgsApplication([], True)
qgs.initQgis()
sys.path.append(r'C:\Program Files\QGIS 3.12\apps\qgis\python\plugins')

In [ ]:
study = QgsProject.instance()

## Import Map Layer

In [ ]:
eu_path = "files/countries/CNTR_BN_10M_2016_3035.shp"
eu_layer = QgsVectorLayer(eu_path, 'EU Map', 'ogr')

if not eu_layer.isValid():
    print("Failed to load the layer!")
else:
    study.addMapLayer(eu_layer)

## Read CSV with Pandas

In [ ]:
import pandas as pd
df = pd.read_csv("files/paises.csv")
df.head(df.shape[0])

## Variation Graphic with Matplotlib

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(25, 7))
plt.bar(df.Pais, df.Value)
plt.xlabel('Paises')
plt.ylabel('% de variação')
plt.title('Diminuição na Emissão de CO2 Proveniente do Setor Elétrico')
plt.show()

In [ ]:
#load csv

valores = ['paises']

def loadCSVVectorLayer(aux):
    val_path = "file:files/{}.csv".format(aux)
    layer_name = '{}'.format(aux)
    val_layer = QgsVectorLayer(val_path, layer_name, 'delimitedtext')

    if not val_layer.isValid():
        print('Layer failed to load!')
    else:
        study.addMapLayer(val_layer)
        
for aux in valores:
    loadCSVVectorLayer(aux)

In [ ]:
targetField = 'FID' # verificar FIDs
joinField = 'pais'

In [ ]:
def joinCSVSHPVectorLayers(csv_layer):
    joinObject = QgsVectorLayerJoinInfo()
    joinObject.setJoinFieldName(joinField)
    joinObject.setTargetFieldName(targetField)
    joinObject.setJoinLayerId(csv_layer.id())
    joinObject.setUsingMemoryCache(True)
    joinObject.setJoinLayer(csv_layer)
    flag = eu_layer.addJoin(joinObject)
    return flag
    
layer_name = 'paises'
csv_layer = study.mapLayersByName(layer_name)[0]
flag = joinCSVSHPVectorLayers(csv_layer)